In [ ]:
import torch
import torch.nn as nn
import torch.utils.data

import torchsummary

from tqdm import tqdm
import pandas as pd
import copy

import plotly.express as ep

from dataloader import ECGDataset

In [ ]:
BATCH_SIZE = 100
LR = 1e-3
EPOCHS = 2


device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
trainset = ECGDataset()
testset = ECGDataset()

trainloader = torch.utils.data.DataLoader(trainset, BATCH_SIZE,True, pin_memory=True)
testloader = torch.utils.data.DataLoader(testset, BATCH_SIZE,True, pin_memory=True)


In [ ]:
class Network(nn.Module):
    def __init__(self,number_of_classes, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.layers = nn.Sequential(
            nn.Conv1d(in_channels=12,out_channels=128,kernel_size=80,stride=64),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(in_channels=128,out_channels=256,kernel_size=11,stride=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Conv1d(in_channels=256,out_channels=128,kernel_size=7,stride=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(),
            nn.Linear(in_features=128,out_features=64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(in_features=64,out_features=number_of_classes),
        )
    def forward(self, x):
        return self.layers(x)

In [ ]:
model = Network(2)
model = model.to(device)


criterum = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(),LR,weight_decay=1e-5) 
optimizer = torch.optim.SGD(model.parameters(),LR*10,momentum=0.9,weight_decay=1e-5) 

lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer,max_lr=LR,total_steps=EPOCHS)

In [ ]:
def train(model, dataloader):
    running_acc = 0
    running_loss = 0
    model.train()
    with tqdm(enumerate(dataloader)) as t:
        t.set_description_str('Train')
        for i,(data,labels) in t:
            # reset
            optimizer.zero_grad(True)

            # Data transfere
            data = data.to(device,non_blocking=True).float()
            labels = labels.to(device,non_blocking=True)

            # network inference
            target = model(data)

            # loss
            loss = criterum(target,labels)
            running_loss += loss.item()

            # backprobagation
            loss.backward()
            optimizer.step()

            # accuracy calculation
            running_acc += (torch.argmax(target,-1)==labels).sum().item()

            t.set_postfix({'loss':running_loss/(BATCH_SIZE*(i+1)),'acc':running_acc/(BATCH_SIZE*(i+1))})
    return {
        'loss':running_loss/(len(dataloader)*BATCH_SIZE),
        'acc':running_acc/(len(dataloader)*BATCH_SIZE),
        }

def test(model, dataloader):
    running_acc = 0
    running_loss = 0
    model.eval()
    with tqdm(enumerate(dataloader)) as t:
        t.set_description_str('Test ')
        for i,(data,labels) in t:
            # Data transfere
            data = data.to(device,non_blocking=True).float()
            labels = labels.to(device,non_blocking=True)

            # network inference
            target = model(data)

            # loss
            loss = criterum(target,labels)
            running_loss += loss.item()

            # accuracy calculation
            running_acc += (torch.argmax(target,-1)==labels).sum().item()

            t.set_postfix({'loss':running_loss/(BATCH_SIZE*(i+1)),'acc':running_acc/(BATCH_SIZE*(i+1))})
    return {
        'loss':running_loss/(len(dataloader)*BATCH_SIZE),
        'acc':running_acc/(len(dataloader)*BATCH_SIZE),
        }



In [ ]:
stats = []

best_test = 0
best_model = None
for i in range(EPOCHS):
    train_dict = train(model,trainloader)
    test_dict = test(model,testloader)

    train_dict = dict([('Train_'+key,value) for key,value in train_dict.items()])
    test_dict = dict([('Test_'+key,value) for key,value in test_dict.items()])


    stats.append(train_dict|test_dict|{'Epoch':i,'LR':lr_scheduler.get_lr()[0]})
    if test_dict['Test_acc']>best_test:
        best_test = test_dict['Test_acc']
        best_model = copy.deepcopy(model)

    lr_scheduler.step()
stats = pd.DataFrame(stats)
stats.to_csv('train_progress.csv')

torch.save(best_model.state_dict(),'best_model_state_dict.pt')
torch.save(best_model,'best_model.pt')


In [ ]:

best_model_load = Network(2)
best_model_load.load_state_dict(torch.load('best_model_state_dict.pt')) 


In [ ]:
torchsummary.summary(model,(12,5000))

In [ ]:
fig = ep.line(stats,x='Epoch',y=['Train_loss','Test_loss'])
fig.show()
fig.write_html('./tmp.html')
fig=ep.line(stats,x='Epoch',y=['Train_acc','Test_acc'])
fig.show()
fig=ep.line(stats,x='Epoch',y=['LR'])
fig.show()